<a href="https://colab.research.google.com/github/millenagena/Challenge-Data-Science-Alura-2ed/blob/main/3-Modelo-de-recomendacao/recomendacao_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semanas 3 e 4

Durante as semanas 3 e 4 vamos focar em melhorar o sistema de recomendação da InsightPlaces.

Nosso sistema não está recebendo os clicks que esperava apenas recomendando imoveis das mesmas regiões e na mesma faixa de preço, por isso como parte do time de Data Science precisamos criar uma nova mecânica de recomendação de imóveis.

O objetivo é criar um sistema de recomendação de imóveis baseado em similaridade de características. A base de dados utilizada será a base de dados tratada na semana 2, após transformar as variáveis categóricas em variáveis binárias e antes de aplicarmos a vetorização.

## Instalando pyspark

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=437ff611c7b1c4414d4b12a2a2a80f5ba9801356f3bc2a4c92aa154231efeb47
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName('recomedacao')\
    .getOrCreate()
    
spark

## Carregamento dos dados

In [3]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip' && unzip semanas-3-e-4.zip -d dados/

--2022-12-19 18:51:27--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.238.155, 52.216.153.100, 52.217.225.145, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.238.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip’

semanas-3-e-4.zip   100%[===================>]   2.47M  --.-KB/s    in 0.1s    

2022-12-19 18:51:27 (23.4 MB/s) - ‘semanas-3-e-4.zip’ saved [2588308/2588308]

Archive:  semanas-3-e-4.zip
   creating: dados/dataset_ml_parquet/
  inflating: dados/dataset_ml_parquet/_SUCCESS  
  inflating: dados/dataset_ml_parquet/._SUCCESS.crc  
  inflating: dados/dataset_ml_parquet/part-00003-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: dados/dataset_ml_parquet/part-00001-a14b227c-f87e-4893-

## Leitura dos dados

In [4]:
dados = spark.read.parquet("dados/dataset_ml_parquet")
dados.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

In [5]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable 

In [6]:
print(f"Quantidade de linhas: {dados.count()}")
print(f"Quantidade de colunas: {len(dados.columns)}")

Quantidade de linhas: 66551
Quantidade de colunas: 25


Vamos verificar a quantidade de dados nulos em cada coluna 

In [7]:
import pyspark.sql.functions as f

In [8]:
dados.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in dados.columns]).show()

+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|condominio|iptu|valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|  0|    0|        0|        0|      0|     0|   0|     0|         0|   0|    0|           0|         0|         0|       0|       0|                 0|            0|                 0|       0|      0|         0|    

## Preparando os dados

Vamos aplicar as transformações necessárias para utilizar os dados no nosso modelos de clustering.

Assim como fizemos em modelos de regressão, é necessário vetorizar os dados com a classe [VectorAssembler](https://spark.apache.org/docs/3.3.1/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html) do PySpark.

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
dados.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

Vamos utilizar todas as colunas, exceto as colunas: 
* `id` - por ser um identificador único; 
* `bairro` - por ser uma variável categórica que não foi transformada em dummy pois possui muitas categorias.

In [11]:
X = dados.columns
X.remove('id')
X.remove('bairro')
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [12]:
vecAssembler = VectorAssembler(inputCols=X, outputCol='features')

In [13]:
dados_vector = vecAssembler.transform(dados).select(['features'])

In [14]:
dados_vector.show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------------------+
|features                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|[2.0,35.0,1.0,1.0,0.0,0.0,100.0,100.0,245000.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]     |
|(23,[0,1,2,3,5,6,7,8,10,15,17,19,20,22],[1.0,84.0,2.0,2.0,1.0,770.0,105.0,474980.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(23,[1,2,3,6,7,8,12,14,17],[85.0,2.0,2.0,460.0,661.0,290000.0,1.0,1.0,1.0])                                 |
|(23,[1,2,3,5,6,7,8,11,18,19],[58.0,1.0,2.0,1.0,550.0,550.0,249000.0,1.0,1.0,1.0])                           |
|(23,[1,2,3,4,5,6,8,10],[64.0,2.0,2.0,1.0,1.0,850.0,530000.0,1.0])                                           |
+------------------------------------------------------------------------------------------------------------+
o

## Padronizando os dados

Vamos padronizar os dados com a classe [StandardScaler](https://spark.apache.org/docs/3.3.1/api/python/reference/api/pyspark.ml.feature.StandardScaler.html) do PySpark. Essa classe é utilizada para padronizar os dados, ou seja, transformar os dados para que tenham média 0 e desvio padrão 1, fórmula:

$$x_{i} = \frac{x_{i} - \mu}{\sigma}$$

In [15]:
from pyspark.ml.feature import StandardScaler

In [16]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dados_vector)
dados_scaler = scaler_model.transform(dados_vector)

In [17]:
dados_scaler.show(5)

+--------------------+--------------------+
|            features|     scaled_features|
+--------------------+--------------------+
|[2.0,35.0,1.0,1.0...|[0.13607726247524...|
|(23,[0,1,2,3,5,6,...|(23,[0,1,2,3,5,6,...|
|(23,[1,2,3,6,7,8,...|(23,[1,2,3,6,7,8,...|
|(23,[1,2,3,5,6,7,...|(23,[1,2,3,5,6,7,...|
|(23,[1,2,3,4,5,6,...|(23,[1,2,3,4,5,6,...|
+--------------------+--------------------+
only showing top 5 rows



## Redução de dimensionalidade

Para fazer a redução da dimensionalidade, ou em outras palavras reduzir o número de colunas, vamos utilizar a classe [PCA](https://spark.apache.org/docs/3.3.1/api/python/reference/api/pyspark.ml.feature.PCA.html) do PySpark. Com isso teremos um novo conjunto de dados onde as colunas serão combinação linear das colunas originais.

In [18]:
from pyspark.ml.feature import PCA

In [19]:
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_scaler)
dados_pca = model_pca.transform(dados_scaler)

In [20]:
dados_pca.select('pca_features').show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533812,1.3380985269405694]  |
|[-3.2529111812184865,-1.1179591836228857]|
|[-1.0611769329629537,-1.6685040058694294]|
|[-1.995811900783872,-1.7655796610847843] |
|[-0.4181406070145923,-2.131040791141693] |
+-----------------------------------------+
only showing top 5 rows



## Extra: Construindo um pipeline

Vamos construir um [pipeline](https://spark.apache.org/docs/3.3.1/api/python/reference/api/pyspark.ml.Pipeline.html) para facilitar a aplicação das transformações nos dados. Podemos definir um pipeline como um conjunto de transformações aplicadas sequencialmente no conjunto de dados.

In [21]:
from pyspark.ml import Pipeline

Para construir um pipeline, precisamos definir um conjunto de estágios, onde cada estágio é uma transformação. No nosso caso, os estágios serão:

* `vector_assembler` - vetorização dos dados;
* `standard_scaler` - padronização dos dados;
* `pca` - redução de dimensionalidade.

In [22]:
pca_pipeline = Pipeline(stages=[vecAssembler, scaler, pca])

In [23]:
pca_pipeline_model = pca_pipeline.fit(dados)

In [24]:
dados_pca = pca_pipeline_model.transform(dados)

In [25]:
dados_pca.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|     scaled_features|        pca_features|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-----------

Selecionando somente a coluna que é o resultado da redução de dimensionalidade, `pca_features`, que é o último estágio do pipeline.

In [26]:
dados_pca.select('pca_features').show(truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533812,1.3380985269405694]  |
|[-3.2529111812184865,-1.1179591836228857]|
|[-1.0611769329629537,-1.6685040058694294]|
|[-1.995811900783872,-1.7655796610847843] |
|[-0.4181406070145923,-2.131040791141693] |
|[-6.9015238649802715,-6.262240376053037] |
|[-0.7164221492881188,-1.4040232409557425]|
|[-1.4856960209870214,-1.7808507073415798]|
|[-6.0456283208921535,-7.824880763294744] |
|[-6.199240571378114,0.312420922354934]   |
|[-6.884681111573803,-1.050675104927925]  |
|[-4.613470387277003,-4.014655897062348]  |
|[-4.476327061805593,-3.8649262122150287] |
|[-0.9935595784851534,-2.9244548608453225]|
|[-3.16451925334317,-2.145186138172189]   |
|[-3.546059940548585,-4.655748957792984]  |
|[-7.507064348067576,-5.873363966077033]  |
|[-0.65020079746698,-0.44347741220645925] |
|[-0.12715321773623325,-1.19611485883027] |
|[-2.849363391484401,-1.04138771

## Criando nossos cluster com KMeans

Cluster é um agrupamento de dados que possuem características semelhantes. O objetivo do [KMeans](https://spark.apache.org/docs/3.3.1/api/python/reference/api/pyspark.ml.clustering.KMeans.html) é encontrar grupos de dados que possuem características semelhantes e coloca-los em um mesmo cluster. 

O KMeans é um algoritmo de aprendizado não supervisionado, ou seja, não é necessário informar a variável alvo para o algoritmo, por essa razão deixamos a coluna `valor` em nosso conjunto de dados. 

In [27]:
from pyspark.ml.clustering import KMeans

O SEED é ser usado para gerar um numero pseudoaleatório. Esse parâmetro é de interesse para nós porque vamos poder reproduzir os nossos resultados reprodutíveis em todas as execuções.

In [28]:
SEED = 1224

Utilizaremos os seguintes parâmetros:

* `featuresCol` - coluna que contém os dados que serão utilizados para a construção dos clusters;

* `predictionCol` - coluna que contém o cluster que cada dado pertence;

* `k` - número de clusters que serão criados;

* `seed` - número usado para gerar números pseudoaleatórios.

In [29]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca', k=5, seed=SEED)

In [30]:
model_kmeans = kmeans.fit(dados_pca)

In [31]:
predictions_kmeans = model_kmeans.transform(dados_pca)

In [32]:
predictions_kmeans.select('pca_features', 'cluster_pca').show(5, truncate=False)

+-----------------------------------------+-----------+
|pca_features                             |cluster_pca|
+-----------------------------------------+-----------+
|[-6.165125049533812,1.3380985269405694]  |4          |
|[-3.2529111812184865,-1.1179591836228857]|1          |
|[-1.0611769329629537,-1.6685040058694294]|1          |
|[-1.995811900783872,-1.7655796610847843] |1          |
|[-0.4181406070145923,-2.131040791141693] |1          |
+-----------------------------------------+-----------+
only showing top 5 rows



## Extra: Avaliando os clusters

Vamos avaliar os clusters de acordo com as informações dos imóveis, como por exemplo: quantidade banheiros, quartos e tamanho do imóvel.

In [33]:
dados_pca\
    .join(predictions_kmeans.select('id', 'cluster_pca'), on='id')\
    .groupBy('cluster_pca')\
    .agg(
        f.count('id').alias('quantidade'),
        f.mean('valor').alias('valor_medio'),
        f.mean('area_util').alias('area_media'),
        f.round(f.mean('quartos'),0).alias('quartos_medio'),
        f.round(f.mean('vaga'), 0).alias('vagas_medio'),
        f.round(f.mean('banheiros'), 0).alias('banheiros_medio'),
        f.round(f.mean('suites'), 0).alias('suites_medio'),
        f.mean('condominio').alias('condominio_medio'),
        f.mean('iptu').alias('iptu_medio'),
    )\
    .orderBy('cluster_pca')\
    .show()

+-----------+----------+------------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|cluster_pca|quantidade|       valor_medio|        area_media|quartos_medio|vagas_medio|banheiros_medio|suites_medio|  condominio_medio|        iptu_medio|
+-----------+----------+------------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|          0|      8400| 1824196.228452381| 160.0922619047619|          3.0|        2.0|            3.0|         2.0| 8416.800952380952| 7670.802738095238|
|          1|     21783| 656653.0182711289| 73.85911031538356|          2.0|        1.0|            2.0|         0.0|1809.1690768030114|1358.0880962218243|
|          2|      3485|5235330.2364418935|359.43185078909613|          4.0|        4.0|            6.0|         4.0|19961.516786226686| 19351.18048780488|
|          3|     13010|1929868.2346656418|158.96879323597233|  

## Extra: Encontrar o numero ideal de componentes

Para encontrar o número ideal de componentes principais que nosso dados devem possuir após a tranformação com o PCA, vamos utilizar o método `explainedVariance` que retorna a variância explicada por cada componente principal. 

In [34]:
pca_pipeline_model.stages[2].explainedVariance

DenseVector([0.2655, 0.1721])

In [35]:
# Colunas do conjunto de dados: 
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [36]:
# Quantidade de colunas existentes
k = len(X)
k

23

In [37]:
dados_pca.select("pca_features").show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-6.165125049533812,1.3380985269405694]  |
|[-3.2529111812184865,-1.1179591836228857]|
|[-1.0611769329629537,-1.6685040058694294]|
|[-1.995811900783872,-1.7655796610847843] |
|[-0.4181406070145923,-2.131040791141693] |
+-----------------------------------------+
only showing top 5 rows



In [38]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_pca.drop('pca_features'))
dados_imoveis_pca = model_pca.transform(dados_pca.drop('pca_features'))

In [39]:
dados_imoveis_pca.select("pca_features").show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------

Somando a variância explicada de cada componente, percebemos que as 23 colunas explicam 100% dos dados.

In [40]:
sum(model_pca.explainedVariance) * 100

100.0

In [41]:
model_pca.explainedVariance[:]

array([2.65457494e-01, 1.72067301e-01, 9.12744818e-02, 5.43737219e-02,
       5.22298117e-02, 4.66203459e-02, 4.42659008e-02, 4.16021312e-02,
       3.46818214e-02, 2.72266440e-02, 2.43947461e-02, 2.01043117e-02,
       1.92103152e-02, 1.76122901e-02, 1.55070658e-02, 1.38947785e-02,
       1.20074124e-02, 1.13351577e-02, 1.01096341e-02, 9.23012235e-03,
       8.89844650e-03, 7.89606599e-03, 4.66992155e-15])

Vamos utilizar o método [cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) para realizar uma soma cumulativa dos valores de variância cada componente principal.

In [42]:
import numpy as np

In [43]:
lista_valores = np.cumsum(model_pca.explainedVariance[:])
lista_valores

array([0.26545749, 0.43752479, 0.52879928, 0.583173  , 0.63540281,
       0.68202316, 0.72628906, 0.76789119, 0.80257301, 0.82979965,
       0.8541944 , 0.87429871, 0.89350903, 0.91112132, 0.92662838,
       0.94052316, 0.95253057, 0.96386573, 0.97397537, 0.98320549,
       0.99210393, 1.        , 1.        ])

Queremos que nossos componentes expliquem até 70% dos dados:

In [44]:
k = sum(lista_valores <= 0.7)
k

6

Logo faz-se nescessário pelo menos 6 componentes principais para obtermos uma variância de 70% dos dados.

In [45]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_pca.drop('pca_features'))
dados_imoveis_pca_final = model_pca.transform(dados_pca.drop('pca_features'))

In [46]:
dados_imoveis_pca_final.select('pca_features').show(truncate=False, n=5)

+--------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------+
|[-6.165125049533812,1.3380985269405696,-1.7052299823820158,-0.5338289630560162,0.08903815478581815,-0.3134396180129953]   |
|[-3.2529111812184865,-1.1179591836228857,-0.2923895841501507,3.195538820020636,0.1528630698219846,1.302893049066197]      |
|[-1.0611769329629537,-1.6685040058694294,-2.30759482783911,0.10553124125788461,-0.06914386452866851,0.7187181899556143]   |
|[-1.995811900783872,-1.7655796610847843,1.3571962498672347,-0.16336519381168974,-0.021305227846957334,0.12808044918214828]|
|[-0.4181406070145923,-2.131040791141693,-0.1026409926497008,2.809916415039177,-0.042956284698168964,-0.09449068767336455] |


In [47]:
sum(model_pca.explainedVariance) *100

68.2023156221161

Rodando novamente o pipeline para obtermos os dados com a quantidade de explicaçao requerida:

In [48]:
pca_pipeline = Pipeline(stages=[vecAssembler, scaler, pca])

In [49]:
model_pca_pipeline = pca_pipeline.fit(dados)

In [50]:
projection = model_pca_pipeline.transform(dados)

In [51]:
projection.select('pca_features').show(truncate=False, n=5)

+--------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------+
|[-6.165125049533812,1.3380985269405696,-1.7052299823820158,-0.5338289630560162,0.08903815478581815,-0.3134396180129953]   |
|[-3.2529111812184865,-1.1179591836228857,-0.2923895841501507,3.195538820020636,0.1528630698219846,1.302893049066197]      |
|[-1.0611769329629537,-1.6685040058694294,-2.30759482783911,0.10553124125788461,-0.06914386452866851,0.7187181899556143]   |
|[-1.995811900783872,-1.7655796610847843,1.3571962498672347,-0.16336519381168974,-0.021305227846957334,0.12808044918214828]|
|[-0.4181406070145923,-2.131040791141693,-0.1026409926497008,2.809916415039177,-0.042956284698168964,-0.09449068767336455] |


**Silhouette Coefficient:**

Silhoutte Score é uma métrica usada para calculara o quão bom está a clusterização utilizada. Esse valor varia de -1 a 1.

1: Significa que os clusters estão bem separados uns dos outros.

0: Significa que os clusters são indiferentes, ou podemos dizer que a distância entre os clusters não é significativa.

-1: Significa que os clusters são atribuídos da maneira errada.

Vamos aumentar a quantidade de cluster para 50, pois temos uma grande quantidade de dados e queremos que os clusters sejam bem definidos.

In [52]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]

evaluator = ClusteringEvaluator(predictionCol='cluster_pca', featuresCol='pca_features', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_score = {}

for i in range(2,51):
    
    KMeans_algo = KMeans(k=i, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)
    
    KMeans_fit = KMeans_algo.fit(projection)
    
    output = KMeans_fit.transform(projection)
    
    score = evaluator.evaluate(output)
    silhouette_score[i] = score


In [53]:
silhouette_score

{2: 0.48064274154867875,
 3: 0.38408547735277593,
 4: 0.3404444639951651,
 5: 0.36198734513292447,
 6: 0.43109156759111156,
 7: 0.46629810296055413,
 8: 0.40653120074621685,
 9: 0.441853181044332,
 10: 0.43115815888565606,
 11: 0.4961329148469282,
 12: 0.5028371915407917,
 13: 0.5216100850200086,
 14: 0.5127861190288592,
 15: 0.5191873637325003,
 16: 0.5381475918926103,
 17: 0.497670485554723,
 18: 0.5291285729592509,
 19: 0.5120490065648051,
 20: 0.48420086559535264,
 21: 0.5090507237145131,
 22: 0.5166667706677477,
 23: 0.5170865930468537,
 24: 0.5094251880412523,
 25: 0.5402510750846394,
 26: 0.4885579990502713,
 27: 0.4901719715314668,
 28: 0.5182027354701573,
 29: 0.4874454363535331,
 30: 0.4910928223928047,
 31: 0.525005220002699,
 32: 0.508068197715153,
 33: 0.5156595873979353,
 34: 0.5125935830656666,
 35: 0.5048023519285596,
 36: 0.4969647676977379,
 37: 0.5126982922151361,
 38: 0.4936473618330673,
 39: 0.4773135604813528,
 40: 0.5044276458992648,
 41: 0.486012690605044,
 42: 

In [54]:
k_best = [key for key, value in silhouette_score.items() if value == max(silhouette_score.values())]
k_best[0]

25

In [55]:
kmeans = KMeans(k=k_best[0], featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)

In [56]:
modelo_kmeans = kmeans.fit(projection)

In [57]:
projetion_kmeans = modelo_kmeans.transform(projection)

In [58]:
projetion_kmeans.select(['pca_features','cluster_pca', "id"]).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|pca_features                                                                                                               |cluster_pca|id                                  |
+---------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|[-6.165125049533812,1.3380985269405696,-1.7052299823820158,-0.5338289630560162,0.08903815478581815,-0.3134396180129953]    |18         |00002dd9-cc74-4809-b5a5-850adf0e7526|
|[-3.2529111812184865,-1.1179591836228857,-0.2923895841501507,3.195538820020636,0.1528630698219846,1.302893049066197]       |22         |0009ca94-2b37-4381-b8b8-773ce0f92444|
|[-1.0611769329629537,-1.6685040058694294,-2.30759482783911,0.10553124125788461,-0.06914386452866851,0.7187181899556143]    |

## Filtrando imóveis do mesmo cluster

Vamos agora verificar quais imóveis pertencem ao mesmo cluster. Assim, podemos identificar os imóveis que possuem características parecidas.

Para isso vamos escolher um ID de um imóvel aleatório:

In [59]:
id_imovel = '0034df72-124a-4383-a89f-a019850a2ba0'

In [60]:
projetion_kmeans.filter(projetion_kmeans.id == id_imovel).select('cluster_pca').show()

+-----------+
|cluster_pca|
+-----------+
|          9|
+-----------+



In [61]:
cluster = projetion_kmeans.filter(projetion_kmeans.id == id_imovel).select('cluster_pca').collect()[0][0]
cluster

9

In [62]:
imoveis_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('bairro', 'id', 'pca_features')
imoveis_recomendadas.show(5)

+--------------------+--------------------+--------------------+
|              bairro|                  id|        pca_features|
+--------------------+--------------------+--------------------+
|Recreio dos Bande...|0034df72-124a-438...|[-4.6134703872770...|
|Recreio dos Bande...|01b872aa-9563-4e6...|[-3.7479325299310...|
|             Taquara|01e94fd8-5fa6-454...|[-2.5093968825911...|
|     Barra da Tijuca|023db788-b55a-428...|[-4.1939429646588...|
|     Barra da Tijuca|026d8d70-5c0b-4d4...|[-2.4200397234310...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



## Calculando distância euclidiana para encontrar as 10 melhores recomendações

Vamos agora calcular a distância euclidiana entre o imóvel escolhido e os demais imóveis do mesmo cluster, para obter os imóveis que apresentam maior similaridade com o imóvel escolhido. Para calcular essa distância, vamos utilizar o método [euclidean](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html) da biblioteca `scipy.spatial.distance`.

In [63]:
id_imovel

'0034df72-124a-4383-a89f-a019850a2ba0'

In [64]:
imovel_procurado = imoveis_recomendadas.filter(imoveis_recomendadas.id == id_imovel)\
                                          .select('pca_features').collect()[0][0]
imovel_procurado                             

DenseVector([-4.6135, -4.0147, 1.3861, 0.2579, -0.0626, 0.3152])

In [65]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

In [66]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)

euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(imovel_procurado, x), FloatType())

imoveis_recomendadas\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')\
    .show(5)

+--------------------+--------------------+----------+
|              bairro|                  id| distancia|
+--------------------+--------------------+----------+
|Recreio dos Bande...|0034df72-124a-438...|       0.0|
|Recreio dos Bande...|e5ec2e28-c81f-47a...|0.17295614|
|Recreio dos Bande...|12012273-e197-488...|0.18009204|
|Recreio dos Bande...|c4585166-c411-4e5...|0.18062168|
|Recreio dos Bande...|acbccbb4-4aec-452...|0.22523937|
+--------------------+--------------------+----------+
only showing top 5 rows



In [67]:
recomendadas = imoveis_recomendadas\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')

recomendadas.show(5)

+--------------------+--------------------+----------+
|              bairro|                  id| distancia|
+--------------------+--------------------+----------+
|Recreio dos Bande...|0034df72-124a-438...|       0.0|
|Recreio dos Bande...|e5ec2e28-c81f-47a...|0.17295614|
|Recreio dos Bande...|12012273-e197-488...|0.18009204|
|Recreio dos Bande...|c4585166-c411-4e5...|0.18062168|
|Recreio dos Bande...|acbccbb4-4aec-452...|0.22523937|
+--------------------+--------------------+----------+
only showing top 5 rows



## Criando uma função recomendadora para ser usada como referencia para o time dev

Juntando as etapas anteriores, como filtro de imóveis do mesmo cluster, cálculo da distância euclidiana, vamos criar uma função que recebe o ID de um imóvel e retorna os 10 imóveis mais próximos como recomendação.

In [68]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)


def recommender(id_imovel, dataframe_kmeans):
    cluster = dataframe_kmeans\
        .filter(dataframe_kmeans.id == id_imovel)\
        .select('cluster_pca')\
        .collect()[0][0]

    imoveis_recomendadas = dataframe_kmeans\
        .filter(dataframe_kmeans.cluster_pca == cluster)

    imovel_procurado = imoveis_recomendadas\
        .filter(imoveis_recomendadas.id == id_imovel)\
        .select('pca_features')\
        .collect()[0][0]

    euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(
        imovel_procurado, x), FloatType())

    colunas_nao_utilizadas = [
        'features', 'scaled_features', 'pca_features', 'cluster_pca', 'distancia']

    recomendadas = imoveis_recomendadas\
        .withColumn('distancia', euclidean_udf('pca_features'))\
        .select([col for col in imoveis_recomendadas.columns if col not in colunas_nao_utilizadas])\
        .orderBy('distancia')

    return recomendadas


Vamos agora testar o recomendador com o ID de um imóvel aleatório:

In [69]:
recommender("0034df72-124a-4383-a89f-a019850a2ba0", projetion_kmeans).select("id", "bairro").show(20, truncate=False)

+------------------------------------+------------------------+
|id                                  |bairro                  |
+------------------------------------+------------------------+
|0034df72-124a-4383-a89f-a019850a2ba0|Recreio dos Bandeirantes|
|e5ec2e28-c81f-47a6-aeff-1288d6773c7b|Recreio dos Bandeirantes|
|12012273-e197-488f-bf81-784cb02ec875|Recreio dos Bandeirantes|
|c4585166-c411-4e57-9095-9b7ac2f5dccc|Recreio dos Bandeirantes|
|acbccbb4-4aec-4527-88fa-7933de023633|Recreio dos Bandeirantes|
|a5ff60b0-bc83-4e98-b56f-3e855b7b3711|Recreio dos Bandeirantes|
|7e96df1b-e77d-49c5-82e2-cc5ac4f6f960|Recreio dos Bandeirantes|
|d155b386-04e3-48e1-9bb1-f6f14a84a675|Recreio dos Bandeirantes|
|dbe16cf4-439c-4812-9233-34507b399d53|Jacarepaguá             |
|612f2a2c-a28a-44ec-b6d1-4d7606932f84|Jacarepaguá             |
|7c3e3897-3734-4a10-bd7f-2efbb28aaef8|Barra da Tijuca         |
|d102b22b-73a1-48d4-b4da-617bfd0e33c6|Recreio dos Bandeirantes|
|4f991db8-b42f-4ac9-97a4-f41d0630ee1e|Re